# intro

# prepare

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load data -from github

## how to read data directly from github

we need to provide the raw link

In [8]:
url = 'https://raw.githubusercontent.com/r1551z/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-31-2020.csv'
datatest = pd.read_csv(url,error_bad_lines=False)

In [9]:
datatest.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Hubei,Mainland China,1/31/2020 23:59,5806,204.0,141.0
1,Zhejiang,Mainland China,1/31/2020 23:59,538,NaN,14.0
2,Guangdong,Mainland China,1/31/2020 23:59,436,NaN,11.0
3,Henan,Mainland China,1/31/2020 23:59,352,2.0,3.0
4,Hunan,Mainland China,1/31/2020 23:59,332,NaN,2.0


## loop read data

In [110]:
#========optain the data list, starting from 0125------------
import datetime
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(1, 66)]
date_list.reverse()
z = pd.to_datetime(date_list)
z1 = z.month.astype(str).str.zfill(2) + '-' + z.day.astype(str).str.zfill(2) + '-'+ z.year.astype(str)
# z1

In [121]:
# two ways of reading the data
def processdata_new(datatest0):
    datatest = datatest0[['Admin2', 'Province_State', 'Country_Region',
                                           'Deaths','Recovered','Active',
                                          'Combined_Key']]
    
    datatest['Combined_Key'] =datatest['Admin2'].fillna('') + ',' + datatest['Province_State'].fillna('') + ',' + datatest['Country_Region']
    datatest=datatest[['Admin2', 'Province_State', 'Country_Region',
                                                   'Deaths','Recovered','Active',
                                                  'Combined_Key']]
    datatest['Deaths'] = datatest['Deaths'].fillna(0)
    datatest['Recovered'] = datatest['Recovered'].fillna(0)
    datatest['Active'] = datatest['Active'].fillna(0)
  
    return datatest

def processdata_old(datatest0):

    
    datatest0['Active'] = datatest0['Confirmed'] - datatest0['Recovered'] - datatest0['Deaths']
    datatest = datatest0[['Province/State', 'Country/Region',
                                                   'Deaths','Recovered','Active']].rename(columns = {'Province/State': 'Province_State',
                                                                                                     'Country/Region': 'Country_Region'})
    datatest['Admin2'] = None
    datatest['Combined_Key'] =datatest['Admin2'].fillna('') + ',' + datatest['Province_State'].fillna('') + ',' + datatest['Country_Region']
    datatest=datatest[['Admin2', 'Province_State', 'Country_Region',
                                                   'Deaths','Recovered','Active',
                                                  'Combined_Key']]
    datatest['Deaths'] = datatest['Deaths'].fillna(0)
    datatest['Recovered'] = datatest['Recovered'].fillna(0)
    datatest['Active'] = datatest['Active'].fillna(0)

  
    return datatest

In [139]:
## create base datasets: 0124 data
i = '01-22-2020'
url = 'https://raw.githubusercontent.com/r1551z/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + i + '.csv'
datatest0 = pd.read_csv(url,error_bad_lines=False)
datatest = processdata_old(datatest0)


datatest['Confirmed'] = datatest['Deaths'] + datatest['Recovered'] +datatest['Active']

data_confirmed = datatest.loc[:, ['Admin2','Province_State','Country_Region','Combined_Key','Confirmed']].rename(columns={'Confirmed': i}).copy(deep=True)

data_Deaths = datatest.loc[:, ['Admin2','Province_State','Country_Region','Combined_Key','Deaths']].rename(columns={'Deaths': i}).copy(deep=True)
data_Recovered = datatest.loc[:, ['Admin2','Province_State','Country_Region','Combined_Key','Recovered']].rename(columns={'Recovered': i}).copy(deep=True)
data_Active = datatest.loc[:, ['Admin2','Province_State','Country_Region','Combined_Key','Active']].rename(columns={'Active': i}).copy(deep=True)


In [140]:
test = z1
for i in test:
    url = 'https://raw.githubusercontent.com/r1551z/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + i + '.csv'
    datatest0 = pd.read_csv(url,error_bad_lines=False)
    
    try:
        datatest = processdata_new(datatest0)
    except:
        datatest = processdata_old(datatest0)
    datatest['Confirmed'] = datatest['Deaths'] + datatest['Recovered'] +datatest['Active']
    dataset1 = datatest.loc[:, ['Deaths', 'Combined_Key']].rename(columns={'Deaths': i})
    dataset2 = datatest.loc[:, ['Recovered', 'Combined_Key']].rename(columns={'Recovered': i})
    dataset3 = datatest.loc[:, ['Active', 'Combined_Key']].rename(columns={'Active': i})
    dataset4 = datatest.loc[:, ['Confirmed', 'Combined_Key']].rename(columns={'Confirmed': i})
    
    # obtain dataset and fillin the nas
    data_confirmed = data_confirmed.merge(dataset4, how = 'outer', on='Combined_Key')    
    data_confirmed.iloc[:, 4:] = data_confirmed.iloc[:, 4:].fillna(0)

    
    
    data_Deaths = data_Deaths.merge(dataset1, how = 'outer', on='Combined_Key')
    data_Deaths.iloc[:, 4:] = data_Deaths.iloc[:, 4:].fillna(0)
    
    data_Recovered = data_Recovered.merge(dataset2, how = 'outer', on='Combined_Key')
    data_Recovered.iloc[:, 4:] = data_Recovered.iloc[:, 4:].fillna(0)
    
    data_Active = data_Active.merge(dataset3, how = 'outer', on='Combined_Key')
    data_Active.iloc[:, 4:] = data_Active.iloc[:, 4:].fillna(0)

    


F:\programs\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [144]:
data_confirmed.iloc[1, 4:]

01-22-2020      0
01-23-2020      0
01-24-2020      1
01-25-2020      2
01-26-2020      2
01-27-2020     80
01-28-2020     91
01-29-2020    111
01-30-2020    114
01-31-2020    139
02-01-2020    168
02-02-2020    191
02-03-2020    212
02-04-2020    228
02-05-2020    253
02-06-2020    274
02-07-2020    297
02-08-2020    315
02-09-2020    326
02-10-2020    337
02-11-2020    342
02-12-2020    352
02-13-2020    366
02-14-2020    372
02-15-2020    375
02-16-2020    380
02-17-2020    381
02-18-2020    387
02-19-2020    393
02-20-2020    395
             ... 
02-27-2020    410
02-28-2020    410
02-29-2020    411
03-01-2020    413
03-02-2020    414
03-03-2020    414
03-04-2020    418
03-05-2020    418
03-06-2020    422
03-07-2020    426
03-08-2020    428
03-09-2020    428
03-10-2020    429
03-11-2020      0
03-12-2020      0
03-13-2020      0
03-14-2020      0
03-15-2020      0
03-16-2020      0
03-17-2020      0
03-18-2020      0
03-19-2020      0
03-20-2020      0
03-21-2020      0
03-22-2020

In [126]:
data_Active

,01-24-2020,Combined_Key
0,24.0,",Hubei,Mainland China"
1,0.0,",Guangdong,Mainland China"
2,0.0,",Zhejiang,Mainland China"
3,0.0,",Beijing,Mainland China"
4,0.0,",Chongqing,Mainland China"
5,0.0,",Hunan,Mainland China"
6,0.0,",Guangxi,Mainland China"
7,0.0,",Shanghai,Mainland China"
8,0.0,",Jiangxi,Mainland China"
9,0.0,",Sichuan,Mainland China"


# update data everyday

pass

# output

In [147]:
data_Deaths.to_csv('JH_data_globalDeath_timeSeries_from0122.csv')
data_Recovered.to_csv('JH_data_globalRecovered_timeSeries_from0122.csv')
data_Active.to_csv('JH_data_globalActive_timeSeries_from0122.csv')
data_confirmed.to_csv('JH_data_globalConfirmed_timeSeries_from0122.csv')